# Azure Anomaly Detector Tests

We test out Azure's anomaly detector using the free bucks on a personal account.


## Authenticating the Client

Before using the container we pass the api key and endpoint location as environment variables.


In [7]:
from azure.cognitiveservices.anomalydetector import AnomalyDetectorClient
from msrest.authentication import CognitiveServicesCredentials
import os


# variables that come from the microsoft page
# this sample assumes you have created an environment variable for your key and endpoint
# heck the Anomaly detector tutorial on how to create them
SUBSCRIPTION_KEY = os.environ["ANOMALY_DETECTOR_KEY"]
ANOMALY_DETECTOR_ENDPOINT = os.environ["ANOMALY_DETECTOR_ENDPOINT"]

client = AnomalyDetectorClient(ANOMALY_DETECTOR_ENDPOINT, 
                               CognitiveServicesCredentials(SUBSCRIPTION_KEY))

## Loading the time series and Request

Having client ready let's import data examples. Then, create a request object and set the granularity 
(or periodicity, i.e.: Granularity.daily) of data points.

In [8]:
import pandas as pd
from azure.cognitiveservices.anomalydetector.models import Point, Request, Granularity


TIME_SERIES_DATA_PATH = "./example-data/request-data.csv"
series = []
data_file = pd.read_csv(TIME_SERIES_DATA_PATH, header=None, encoding='utf-8', parse_dates=[0])
for index, row in data_file.iterrows():
    series.append(Point(timestamp=row[0], value=row[1]))

request = Request(series=series, granularity=Granularity.daily)

## Using the anomality detector in an entire data point


In [10]:
from azure.cognitiveservices.anomalydetector.models import APIErrorException


print('Detecting anomalies in the entire time series.')
response = None
try:
    response = client.entire_detect(request)
except Exception as e:
    if isinstance(e, APIErrorException):
        print('Error code: {}'.format(e.error.code), 'Error message: {}'.format(e.error.message))
    else:
        print(e)

freaks = [index for index, value in enumerate(response.is_anomaly) if value]


# print the time-series with highlight in anomalies
# it seems easier to plot from the DataFrame than the Point series
data_file.head()


Detecting anomalies in the entire time series.
An anomaly was detected at index:
3
18
21
22
23
24
25
28
29
30
31
32
35
44


,0,1
0,2018-03-01 00:00:00+00:00,32858923
1,2018-03-02 00:00:00+00:00,29615278
2,2018-03-03 00:00:00+00:00,22839355
3,2018-03-04 00:00:00+00:00,25948736
4,2018-03-05 00:00:00+00:00,34139159
